In [17]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob


In [18]:
IMAGESHAPE = [224, 224, 3]
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test'


In [23]:
classes = glob('chest_xray/train/*')
print(classes)

['chest_xray/train\\NORMAL', 'chest_xray/train\\PNEUMONIA']


In [24]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)


In [25]:
for each_layer in vgg_model.layers:
    each_layer.trainable = False


In [6]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)


In [7]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [9]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                shear_range = 0.2,
                                zoom_range = 0.2,
                                horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)


In [10]:
training_set = train_datagen.flow_from_directory('chest_xray/train',
                                                target_size = (224, 224),
                                                batch_size = 4,
                                                class_mode = 'categorical')


Found 5216 images belonging to 2 classes.


In [11]:
test_set = testing_datagen.flow_from_directory('chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 4,
                                            class_mode = 'categorical')


Found 624 images belonging to 2 classes.


In [13]:
fitted_model = final_model.fit(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)


Epoch 1/5
1304/1304 [==============================] - 383s 293ms/step - loss: 0.2927 - accuracy: 0.9260 - val_loss: 0.6355 - val_accuracy: 0.8846
Epoch 2/5
1304/1304 [==============================] - 380s 291ms/step - loss: 0.2992 - accuracy: 0.9452 - val_loss: 0.4655 - val_accuracy: 0.9279
Epoch 3/5
1304/1304 [==============================] - 370s 284ms/step - loss: 0.2185 - accuracy: 0.9557 - val_loss: 0.5780 - val_accuracy: 0.9119
Epoch 4/5
1304/1304 [==============================] - 369s 283ms/step - loss: 0.2926 - accuracy: 0.9490 - val_loss: 0.9404 - val_accuracy: 0.8942
Epoch 5/5
1304/1304 [==============================] - 371s 285ms/step - loss: 0.1969 - accuracy: 0.9611 - val_loss: 0.6800 - val_accuracy: 0.9119


In [14]:
final_model.save('Pneumonia_model.h5')

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('Pneumonia_model.h5') #Loading our model
img=image.load_img('D:/Engineering/Projects/Pneumonia Prediction/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg',target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
    print('Person is safe.')
else:
    print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')


1/1 [==============================] - 0s 174ms/step
Person is affected with Pneumonia.
Predictions: [[8.814389e-12 1.000000e+00]]


In [12]:
train_dir = 'D:/Engineering/Projects/Pneumonia Prediction/chest_xray/train'
class_labels = sorted(os.listdir(train_dir))
print(class_labels)

['NORMAL', 'PNEUMONIA']


In [14]:
true_label = os.path.basename(os.path.dirname(img_path)).upper()
print(true_label)

PNEUMONIA


In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import os

# Define the test image path
img_path = 'D:/Engineering/Projects/Pneumonia Prediction/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg'

# Load the model
model = load_model('Pneumonia_model.h5')

# Load the test image
img = image.load_img(img_path, target_size=(224,224))
img_arr = image.img_to_array(img)
img_arr = np.expand_dims(img_arr, axis=0)
img_data = preprocess_input(img_arr)

# Make a prediction
preds = model.predict(img_data)
class_idx = np.argmax(preds, axis=1)

# Extract class labels from the training data directory
train_dir = 'D:/Engineering/Projects/Pneumonia Prediction/chest_xray/train'
class_labels = sorted(os.listdir(train_dir))

# Define the true and predicted labels
true_label = os.path.basename(os.path.dirname(img_path)).upper()
predicted_label = class_labels[class_idx[0]]

# Calculate confusion matrix and classification report
cm = confusion_matrix([true_label], [predicted_label], labels=class_labels)
cr = classification_report([true_label], [predicted_label], labels=class_labels, target_names=class_labels,zero_division=0)
# Print results
print(f'Prediction: {predicted_label}')
print(f'Confusion matrix:\n {cm}')
print(f'Classification report:\n {cr}')


1/1 [==============================] - 0s 151ms/step
Prediction: PNEUMONIA
Confusion matrix:
 [[0 0]
 [0 1]]
Classification report:
               precision    recall  f1-score   support

      NORMAL       0.00      0.00      0.00         0
   PNEUMONIA       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         1
   macro avg       0.50      0.50      0.50         1
weighted avg       1.00      1.00      1.00         1

